In [ ]:
# Используем pipeline как высокоуровневый помощник
from transformers import pipeline

# Создаем сообщения в формате диалога
messages = [
    {"role": "user", "content": "def print_hello():"},
]

# Используем модель Phi-4-mini-instruct
pipe = pipeline("text-generation", model="microsoft/Phi-4-mini-instruct", trust_remote_code=True)

try:
    # Генерируем ответ на основе сообщений
    response = pipe(messages)
    print(response[0]['generated_text'])
except Exception as e:
    print(f"Произошла ошибка при генерации текста: {e}")
    print("Попробуйте использовать другую модель, например 'sberbank-ai/rugpt3small_based_on_gpt2'")

In [ ]:
import pandas as pd
df = pd.read_parquet('docs/dataset.parquet')
print(df.to_dict())

In [1]:
# Импортируем библиотеку SymSpell для проверки орфографии
from symspellpy import SymSpell, Verbosity
import os
import pandas as pd

# Инициализируем SymSpell с правильными параметрами
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Загружаем словарь для русского языка
# SymSpell использует словарь частотности слов для работы
dictionary_path = "dictionary_ru.txt"

# Загружаем словарь в SymSpell
if not sym_spell.load_dictionary(dictionary_path, encoding='utf-8', term_index=0, count_index=1):
    print(f"Ошибка загрузки словаря из {dictionary_path}")

# Функция для проверки орфографии
def check_spelling(word):
    # Получаем предложения для исправления
    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
    
    if suggestions:
        return f"Возможно, вы имели в виду: {suggestions[0].term} (расстояние: {suggestions[0].distance})"
    return "Исправлений не найдено. Возможно, префикс слишком короткий или словарь не содержит подходящих слов."

# Проверяем ввод пользователя
user_input = "звявка"
print(check_spelling(user_input))


Возможно, вы имели в виду: заявка (расстояние: 1)


In [14]:
import requests

# URL для получения списка всех документов
url = "http://localhost:7777/query" 

# Параметры запроса
params = {
    "query": "звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки звявки "
}

# Выполняем GET-запрос
response = requests.get(url, params=params)

# Выводим ответ
print(response.json())  


{'answer': 'По вашему запросу найдена следующая информация: \n\nЗаявка в статусе "Редактирование", сколько ждать рассмотрения заявки. Статус заявки "Редактирование" означает, что заявка рассмотрена модератором и по причине наличия незначительных ошибок направлена на доработку. Следует перейти в блок "Оферты и СТЕ" - "Мои заявки на СТЕ", очистить поле "Статус" и нажать кнопку "Поиск". Далее нажать на номер нужной заявки и прокрутить заявку вниз. В левом нижнем углу заявки нажать "Редактировать", скорректировать сведения заявки в соответствии с комментарием модератора и направить заявку повторно на рассмотрение (нажав кнопку в правом нижнем углу заявки).\n\nИсточник: Заявка в статусе "Редактирование", сколько ждать рассмотрения заявки\n\nДополнительная информация:\n\nЗаявка в статусе "Черновик", сколько ждать рассмотрения заявки?. Статус заявки "Черновик" означает, что данная заявка создана, но не отправлена на рассмотрение модераторам.\nНеобходимо проверить сведения заявки СТЕ и, в случ